<h1><center>基于机器学习的文本情感分类</center></h1>

> - 语言推荐：Python
> - 设计组成：数据采集与分析、机器学习算法原理、详细的可视化结果

# 01 数据整理与清洗

在本 Notebook 中，我们介绍本次项目的数据来源，并对数据进行整理与清洗，最终得到一个基本可用于机器学习模型训练的文本情感分类数据集。

## 1.1 数据来源介绍

本次项目的数据集来源于作者早些时间进行的项目《基于微博文本的省域用户心理健康状态分析》所采集的微博数据集。

该微博数据集包含 **2.1万用户** 及其发布的约 **50条原创微博文本**，并基于微博文本内容，使用 **大语言模型**（主要为 DeepSeek API）完成对微博的情感分析，使用的相关提示词节选如下：
> 你是一个资深社会心理学家，擅长从社交媒体中识别微妙的情感变化。下面，请基于用户微博数据执行以下分析：
> 1. 综合情感分析：
>    - 识别用户微博中的 **情感类型**，使用"**情感类别（积极/消极/中立/混合）**-子类（如：喜悦/愤怒/哀愁/迷茫）"的格式进行描述
>    - 分析情感的强度（0-10分）
>    - 对微博主体事件进行概述，不超过10个字
>    - 给出微博的关键词，不超过5个
> 2. 方面级情感分析（可选）：
>    - 识别微博中情感，同样使用"情感类别-子类"的格式进行描述
>    - 归纳触发该情感的事件，不超过5个字
>    - 以情感为键、事件为值，构建一个字典。若没有方面级情感，则返回空字典
>
> ...（后略）

数据集以 JSON 形式保存于 `data/original/user_weibo_analysis_data.json` 文件中。

首先，我们先将该数据集以 Python 字典的形式读取至内存。

In [ ]:
import json

with open("data/original/user_weibo_analysis_data.json", "r", encoding="utf-8") as f:
    user_weibo_data: dict = json.load(f)

## 1.2 数据整理

对于本次文本情感分类项目，我们只需要重点关注 **微博文本** 和 **情感类型** 两个字段，此外保留微博发布**用户昵称**用于后续的数据清洗，其他字段可以暂时忽略。

其次，由上述提示词可知：
- 综合情感分析中，情感类别分为“积极”、“消极”、“中立”、“混合”四大类。
    - 其中“混合”类别属于较为复杂的情感状态，对于多数机器学习模型而言，学习难度较大，故在此将类别标注为“混合”的微博从数据集中剔除，然后取“情感类型”中的“情感类别”作为微博文本的最终情感标签，任务简化为 **“积极”“中立”“消极”的三分类问题**
- 原数据集为提高情感分析的细粒度，设置了“方面级情感”的模型分析可选项，同样属于较为复杂的情感状态，故将包含了方面级情感的微博从数据集中剔除
- 为便于处理与训练，仅保留 **文本内容长度在3至50字符之间** 的微博文本

经过上述的处理，可以将原项目的微博数据集转化为本次项目的基础数据集。

In [12]:
# 提取有用的属性作为表头
total_weibos = [("user_name", "text", "sentiment_type")]

for info in user_weibo_data.values():
    user_name = info["screen_name"]
    for w in info["weibos"]:
        text = w["text"]
        sentiment_type = w["sentiment_analysis"]["comprehensive_sentiment"]["type"].replace("中性", "中立").split('-')[0]
        has_asp_sentiment = len(w["sentiment_analysis"]["aspect_sentiment"]) > 0
        if (
            3 < len(text) < 50
            and not has_asp_sentiment
            and "混合" not in sentiment_type
        ):
            total_weibos.append((user_name, text, sentiment_type))

weibo_count = len(total_weibos)
print(f"获得 {weibo_count} 条微博数据")

获得 367322 条微博数据


可以看到，处理后的数据集中包含 **约37万条** 微博文本及其对应的情感标签，数据量较为充足，适合用于后续的机器学习模型训练与评估。

下面将 `total_weibos` 转换为 Python 的 Pandas `DataFrame` 对象，以便后续处理。

In [19]:
from pandas import DataFrame

def convert_to_dataframe(data):
    """
    将处理后的微博数据转换为 Pandas DataFrame

    Args:
        weibo_data (list): 处理后的微博数据列表

    Returns:
        pd.DataFrame: 转换后的 DataFrame
    """
    columns = data[0]
    data_rows = data[1:]
    df = DataFrame(data_rows, columns=columns)
    return df

df = convert_to_dataframe(total_weibos)

df.sample(5)

,user_name,text,sentiment_type
286008,Seeingyouhappy_,加班成了日常的事 只有一味加量超量的工作内容 没有明确员工加班的说辞和补贴 呵~,消极
342740,Just_badadatin,转载了@jjjustin0219 的微博图片,中立
304405,还会记得夏天吗,虽然但是 好讨厌喜欢的二次元角色真人化,消极
298495,狐小十二月,这歹毒的染色。。。怎么会出自我手🙈,消极
69801,桬菻,目之所及，一半银沙，一半湛蓝，一场心动💙 克孜勒苏·白沙山-白沙湖,积极


## 1.3 数据清洗

微博作为一种社交媒体平台，用户发布的内容质量参差不齐，存在大量低质量微博文本，这些文本对于情感分类任务的影响较大，可能导致模型训练效果不佳。

因此，我们需要对数据集进行进一步的清洗工作。

我们认为，表达自己的 **情绪状态、心理想法** 等的微博属于高质量微博，

而那些仅包含广告、抽奖、营销等内容的微博对于情感分析来说价值较低，可以直接将其从数据集中删除。

此外，微博中包含的一些提示语，如“分享图片”“@（用户）”以及外部链接，对于用户表达情感的作用并不大，虽不至于将整条微博删除，但需要将这些提示语从微博文本中剔除。这部分工作主要通过正则表达式完成。

In [20]:
import re

def is_valid(text: str) -> bool:
    """
    判断该微博是否为有效微博，依据为广告、转载、营销相关的关键词
    如果微博中出现了关键词，则认为是低质量微博，直接删除

    Args:
        text (str): 一条微博的文本内容

    Returns:
        bool: 微博是否是否有效
    """

    if len(text.strip()) <= 3:
        return False
    
    # 广告、转载相关关键词
    ad_keywords = [
        '点开红包', '现金红包', '好礼', '网页链接', 
        '我在参与', '连续签到', '粉打卡', '年度歌曲', 
        '免费围观', '关注超话', "蚂蚁庄园：", "森林驿站", 
        "头条文章", "注册微博", "注册微博", "闲鱼发布",
        "闲鱼号", "头像挂件"
    ]

    # 微博中含有任意一个关键词，则认为无效
    if any(kw in text for kw in ad_keywords):
        return False
    
    return True



def clean_text(text, user_name=''):
    """清洗微博文本内容，去除无用信息

    Args:
        text (str): 微博文本内容
        user_name (str, optional): 用户名。 Defaults to ''.

    Returns:
        str: 清洗后的微博文本
    """

    # 去除分享来源信息（直接将“分享自”及其后面的所有信息去除）
    text = re.sub(r'(?:[（(])?分享自(?!己).*$', '', text)

    # 去除“分享图片/视频”文本
    text = re.sub(r'分享(图片|视频)', '', text)
    
    # 去除多余空格
    text = re.sub(r'[\s]+', ' ', text)

    # 去除话题标签
    text = re.sub(r'#.*?#', '', text)

    # 去除视频来源信息
    text = re.sub(rf'{re.escape(user_name)}的微博视频\s*', '', text)

    # 去除@用户名
    text = re.sub(r'@[\w\-]+', '', text)

    # 去除URL
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # 去除特殊字符与多余标点
    text = re.sub(r'[^\u4e00-\u9fff\w\s，。！？；：""''()（）【】]', '', text)
    
    # 去除纯表情符号的内容
    if len(re.sub(r'[\s]', '', text)) <= 1:
        return ''
    
    return text.strip()

In [21]:
weibo_cleaned = [total_weibos[0]]

for w in total_weibos[1:]:
    user_name, text, sentiment_type = w
    text = clean_text(text, user_name)
    if is_valid(text):
        weibo_cleaned.append((user_name, text, sentiment_type))

weibo_cleaned_count = len(weibo_cleaned)
print(f"清洗后获得 {weibo_cleaned_count} 条微博数据，保留率为 {weibo_cleaned_count / weibo_count:.2%}")

清洗后获得 358585 条微博数据，保留率为 97.62%


In [22]:
df_cleaned = convert_to_dataframe(weibo_cleaned)

df_cleaned.sample(5)

,user_name,text,sentiment_type
167600,是憨不拉多,没想到我还能被听泉迷住哈哈哈,积极
271327,糖霜森林,我靠后悔。。。感觉自己是个孙子似的。。。 违背自己内心了吗 性格变了吗。。。,消极
119532,如我所袁33发大财,真的不懂那些某书达人看着BF简要都能写漏的东西？？？？？？失踪？,消极
221121,就不爱吃年糕,饮料测评,中立
97895,XueXkang,快乐的是无数寻求到共鸣的瞬间,积极


经过清洗，`df_cleaned` 中的 `user_name` 列对于后续任务的作用已经不大，可以将其删除，仅保留 `weibo_text` 和 `sentiment_type` 两列。

最后，将清洗好的数据以 CSV 格式保存至 `data/preparation/weibo_cleaned.csv` 文件中，供后续使用。

In [ ]:
df_cleaned = df_cleaned.drop(columns=['user_name'])

def save_to_csv(df: DataFrame, file_path: str):
    """将 DataFrame 保存为 CSV 文件"""

    # 使用 utf-8-sig 编码以支持 Excel 正确显示中文
    df.to_csv(file_path, index=False, encoding='utf-8-sig')  

save_to_csv(df_cleaned, "data/preparation/weibo_cleaned.csv")